In [1]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt 
import jupyros as jr
import rospy as rp
import actionlib
import actionlib.msg
import assignment_2_2024.msg
import math

In [2]:
from std_srvs.srv import SetBool
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
from actionlib_msgs.msg import GoalStatus
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation
from IPython.display import display
from matplotlib.ticker import MaxNLocator

In [3]:
if not rp.core.is_initialized():
    rp.init_node("target_client", anonymous=True)

In [4]:
obstacle_distances = []
x_data = []
y_data = []
target_setted = False

In [5]:
def init_anim():
    ln_anim.set_data([], [])
    return ln_anim,

def update_anim(frame):
    if len(x_data) > 0:
        ln_anim.set_data(x_data, y_data)
    return ln_anim,

In [6]:
x_input = widgets.BoundedFloatText(description='Target X:', value=0.0, min=-8.0, max=8.0)
y_input = widgets.BoundedFloatText(description='Target Y:', value=0.0, min=-8.0, max=10.0)

send_button = widgets.Button(description='Send Goal')
cancel_button = widgets.Button(description='Cancel Goal')
show_result_button = widgets.Button(description='Show Goal Results')
distance_display_button = widgets.Button(description='Show Obastacle Distance')

distance_text = widgets.FloatText(description="Distance (m):", disabled=True)
output = widgets.Output()
output2 = widgets.Output()
out = widgets.Output(layout={'border': '1px solid black'})

In [7]:
def odom_callback(msg): 
    global x_data, y_data
    y_data.append(msg.pose.pose.position.y) 
    x_data.append(msg.pose.pose.position.x)
def calculate_obs_distance(msg):
    global obstacle_distances
    valid_ranges = [r for r in msg.ranges if not np.isnan(r) and r > 0]
    if valid_ranges:
        closest = min(valid_ranges)
        obstacle_distances.append(closest)
def plot_goal_results(counts):
    labels = ['Reached', 'Not Reached']
    values = [counts['reached'], counts['not_reached']]
    colors = ['green', 'red']
    
    fig_result, ax_result = plt.subplots()
    ax_result.bar(labels, values, color=colors)
    ax_result.set_title("Reached vs Not-Reached Targets")
    ax_result.set_ylabel("Count")
    
    ax_result.yaxis.set_major_locator(MaxNLocator(integer=True))
    plt.show()

In [8]:
def on_send_clicked(b):
    global target_setted
    with output:
        output.clear_output()
        goal_handler.set_goal(x_input.value, y_input.value)
    with out:
        print("New Goal has been sended")
        target_setted = True
def on_cancel_clicked(b):
    global target_setted
    with output:
        output.clear_output()
        goal_handler.cancel_goal()
    with out:
        if target_setted:
            print("A goal has been deleted")
            target_setted = False
        else:
            print("There's no goal to delete")
def on_distance_button_clicked(b):
    if obstacle_distances:
        distance_text.value = round(obstacle_distances[-1], 2)
    else:
        distance_text.value = -1
def on_show_result_clicked(b):
    with output2:
        output2.clear_output()
        plot_goal_results(goal_handler.goal_counts)


In [9]:
class GoalHandler:
    def __init__(self):
        self.remove_target = True
        self.goal_counts = {"reached": 0, "not_reached": 0}
        
        self.client = actionlib.SimpleActionClient("/reaching_goal", assignment_2_2024.msg.PlanningAction)
        self.client.wait_for_server()
        self.send_done_cb = self.done_callback

        jr.subscribe('/odom', Odometry, odom_callback)
        jr.subscribe("/scan", LaserScan, calculate_obs_distance)

    def set_goal(self, x, y):
        if not self.remove_target:
            self.client.cancel_goal()
            self.goal_counts["not_reached"] += 1
            rp.loginfo("Previous goal was overwritten.")

        goal = assignment_2_2024.msg.PlanningGoal()
        goal.target_pose.pose.position.x = x
        goal.target_pose.pose.position.y = y

        rp.set_param("/des_pos_x", x)
        rp.set_param("/des_pos_y", y)

        self.client.send_goal(goal, done_cb=self.send_done_cb)
        self.remove_target = False
        print("A new goal has been set")
        
    def cancel_goal(self):
            if not self.remove_target:
                self.client.cancel_goal()
                self.remove_target = True
                rp.loginfo("Goal canceled")
            else:
                rp.logwarn("There's no goal to delete")

    def done_callback(self, status, result):
        if status == GoalStatus.SUCCEEDED:
            self.goal_counts["reached"] += 1
        else:
            self.goal_counts["not_reached"] += 1

In [10]:
goal_handler = GoalHandler()

In [11]:
send_button.on_click(on_send_clicked)
cancel_button.on_click(on_cancel_clicked)
distance_display_button.on_click(on_distance_button_clicked)
show_result_button.on_click(on_show_result_clicked)

In [12]:
display(widgets.HBox([x_input, y_input]))
display(widgets.HBox([send_button, cancel_button, ]))
display(widgets. HBox([distance_display_button, distance_text]))
display(show_result_button)
display(output)
display(output2)
out

Button(description='Show Goal Results', style=ButtonStyle())

Output()

Output()

Output(layout=Layout(border='1px solid black'))

In [13]:
xdata_anim = []
ydata_anim = []

path_output = widgets.Output()
display(path_output)

with path_output:
    fig_anim, ax_anim = plt.subplots()
    ln_anim, = ax_anim.plot([], [], 'ro-')
    ax_anim.set_xlim((-10, 10))
    ax_anim.set_ylim((-10, 10))
    ax_anim.set_title("Robot Path")
    ax_anim.set_xlabel("x")
    ax_anim.set_ylabel("y")

Output()

In [14]:
ani = FuncAnimation(fig_anim, update_anim, init_func=init_anim,
                    interval=500, blit=True)